In [5]:
%load_ext autoreload
%autoreload 2
import os
import sys
import shutil
import argparse
import time
import json
from datetime import datetime
from collections import defaultdict
from itertools import islice
import pickle
import copy
import numpy as np
import cv2
from easydict import EasyDict
from IPython.core.debugger import set_trace
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import autograd
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.parallel import DistributedDataParallel
from tensorboardX import SummaryWriter

from mvn.models.triangulation import RANSACTriangulationNet, AlgebraicTriangulationNet, VolumetricTriangulationNet
from mvn.models.volumetric_temporal import VolumetricTemporalNet,\
                                           VolumetricTemporalAdaINNet,\
                                           VolumetricFRAdaINNet
from mvn.models.v2v import AdaIN, V2VModel, Basic3DBlock, Res3DBlock, Upsample3DBlock, Pool3DBlock, EncoderDecorder
from mvn.models.loss import KeypointsMSELoss, KeypointsMSESmoothLoss, KeypointsMAELoss, KeypointsL2Loss, VolumetricCELoss
from mvn.utils import img, multiview, op, vis, misc, cfg
from mvn.datasets.human36m import Human36MTemporalDataset, Human36MMultiViewDataset
from mvn.datasets import utils as dataset_utils

from mvn.utils.multiview import project_3d_points_to_image_plane_without_distortion
from mvn.utils.vis import draw_2d_pose
from mvn.utils import img
from mvn.utils import volumetric
from mvn.utils import op
from mvn.utils import vis
from mvn.utils import misc
from mvn.utils import cfg
from mvn.datasets import utils as dataset_utils
from mvn.utils.img import image_batch_to_numpy, denormalize_image,to_numpy


from torch.nn.utils.rnn import pack_padded_sequence
from pytorch_convolutional_rnn.convolutional_rnn import Conv2dLSTM, Conv2dPeepholeLSTM

from train import setup_human36m_dataloaders

device = torch.cuda.current_device()

def get_n(m):
    s = 0
    for param in m.parameters():
        s+=param.nelement()
    print (m.__class__, s//1024)    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
config_path = './experiments/human36m/train/human36m_backbone.yaml'
config = cfg.load_config(config_path)
train_loader, val_loader, _ = setup_human36m_dataloaders(config,
                                                         is_train=True,
                                                         distributed_train=False)

for batch in train_loader:
    break
    
images_batch, keypoints_3d_batch_gt, keypoints_3d_validity_batch_gt, proj_matricies_batch = dataset_utils.prepare_batch(batch, device)

In [29]:
grid, cuboids, base_points = op.get_coord_volumes(kind='mpii', 
                                                    training=True, 
                                                    rotation=False,
                                                    cuboid_side=2500, 
                                                    volume_size=64, 
                                                    device=device, 
                                                    keypoints = keypoints_3d_batch_gt,
                                                    batch_size = None,
                                                    dt = None)

In [26]:
batch_index= 0
grid[batch_index,-1,-1,-1]

tensor([241.5050,   1.1129, 729.7400], device='cuda:0')

In [28]:
grid[batch_index,-1,-1,-1] - grid[batch_index,-1,-1,0] 

tensor([ 0.,  0., 10.], device='cuda:0')